<a href="https://colab.research.google.com/github/quinn-dougherty/DS-Unit-2-Sprint-2-Linear-Regression/blob/master/module3-gradient-descent/ASSGNMNT_Gradient_Descent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gradient Descent Implementation Challenge!!

## Use gradient descent to find the optimal parameters of a **multiple** regression model. (We only showed an implementation for a bivariate model during lecture.)

A note: Implementing gradient descent in any context is not trivial, particularly the step where we calculate the gradient will change based on the number of parameters that we're trying to optimize for. You will need to research what the gradient of a multiple regression model looks like. This challenge is pretty open-ended but I hope it will be thrilling. Please work together, help each other, share resources and generally expand your understanding of gradient descent as you try and achieve this implementation. 

## Suggestions:

Start off with a model that has just two $X$ variables You can use any datasets that have at least two x variables. Potential candidates might be the blood pressure dataset that we used during lecture on Monday: [HERE](https://college.cengage.com/mathematics/brase/understandable_statistics/7e/students/datasets/mlr/excel/mlr02.xls) or any of the housing datasets. You would just need to select from them the two varaibles $x$ variables and one y variable that you want to work with that you most want to work with. 

Use Sklearn to find the optimal parameters of your model first. (like we did during the lecture.) So that you can compare the parameter estimates of your gradient-descent linear regression to the estimates of OLS linear regression. If implemented correctly they should be nearly identical.

Becoming a Data Scientist is all about striking out into the unknown, getting stuck and then researching and fighting and learning until you get yourself unstuck. Work together! And fight to take your own learning-rate fueled step towards your own optimal understanding of gradient descent! 


In [294]:
##
### https://web.stanford.edu/~hastie/ElemStatLearn/

'''Prostate data info

Predictors (columns 1--8)

lcavol
lweight
age
lbph
svi
lcp
gleason
pgg45

outcome (column 9)

lpsa

train/test indicator (column 10)

This last column indicates which 67 observations were used as the 
"training set" and which 30 as the test set, as described on page 48
in the book.

There was an error in these data in the first edition of this
book. Subject 32 had a value of 6.1 for lweight, which translates to a
449 gm prostate! The correct value is 44.9 gm. We are grateful to
Prof. Stephen W. Link for alerting us to this error.

The features must first be scaled to have mean zero and  variance 96 (=n)
before the analyses in Tables 3.1 and beyond.  That is, if x is the  96 by 8 matrix
of features, we compute xp <- scale(x,TRUE,TRUE)

'''
import pandas as pd
import altair as alt
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from functools import reduce


def repeatedly(f, n): 
  def composition(f,g):
    # "f after g" 
    return lambda x: f(g(x))
  return reduce(lambda ma, mma: composition(ma, mma), [f]*n)

df = pd.read_csv("https://web.stanford.edu/~hastie/ElemStatLearn/datasets/prostate.data", 
                 delim_whitespace=True)

df['ones'] = np.ones(df.shape[0])

sc = StandardScaler()

#df.select_dtypes(include=['float', 'int']).shape, df.shape

df_normalized = pd.DataFrame(sc.fit_transform(df.select_dtypes(include=['float', 'int'])), columns=df.select_dtypes(include=['float', 'int']).columns)

df_normalized.head()


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45,lpsa,ones
0,-1.645861,-2.016634,-1.872101,-1.030029,-0.525657,-0.867655,-1.047571,-0.868957,-2.533318,0.0
1,-1.999313,-0.725759,-0.791989,-1.030029,-0.525657,-0.867655,-1.047571,-0.868957,-2.299712,0.0
2,-1.587021,-2.200154,1.368234,-1.030029,-0.525657,-0.867655,0.344407,-0.156155,-2.299712,0.0
3,-2.178174,-0.812191,-0.791989,-1.030029,-0.525657,-0.867655,-1.047571,-0.868957,-2.299712,0.0
4,-0.510513,-0.461218,-0.251933,-1.030029,-0.525657,-0.867655,-1.047571,-0.868957,-1.834631,0.0


In [0]:
def LR(X, y, testsize=0.3): 
  #print(X.shape, y.shape)
  # X is a dataframe with arbitrary features
  # y is a dataframe with one feature
  # they are each sliced from the same master df. 
  
  # Split into test and train datasets
  X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=testsize, random_state=42)

  # fit model using train datasets
  model = LinearRegression()
  model.fit(X_train, Y_train)

  # Create new predictions using x_test
  y_pred = model.predict(X_test)

  # Measure Accuracy using y_test and y_pred
  RMSE = (np.sqrt(mean_squared_error(Y_test, y_pred)))
  R2 = r2_score(Y_test, y_pred)

  print('RMSE is {}'.format(RMSE))
  print('R^2 is {}'.format(R2))

  print("coefficients: ", model.coef_)
  print("intercepts: ", model.intercept_)
  
  return {'RMSE': RMSE, 'R2': R2, 'coefficients': model.coef_, 'intercept': model.intercept_}

In [322]:
X = df[['ones', 'lweight']]
y = df.lpsa

from_sklearn = LR(X.drop('ones', axis=1),y)


RMSE is 0.9707535248091201
R^2 is 0.19086153418651874
coefficients:  [1.12248257]
intercepts:  -1.603690194602804


In [324]:

class GDOLSR: 
  def __init__(self, df, feat, target, epochs=2000, gamma=0.11):
    self.df = df
    self.X = df[['ones', feat]]
    self.y = df[target]
    self.theta = np.ones(X.shape[1])[:, np.newaxis]
    self.epochs = epochs
    self.gamma = gamma
    self.N = self.X.shape[1]
    self.feat = feat
    #self.X[feat] = self.normalize(feat)
    
  def normalize(self, feat):
    self.X[feat] = (self.X[feat] - self.X[feat].mean()) / self.X[feat].std()

  def yhat(self, theta): 
    # long form: X.ones1 * theta[0] + X.lcavol * theta[1]
    return pd.DataFrame(np.matmul(self.X, theta))

  def cost(self, theta): 
    sqrd_residuals = (self.y[:, np.newaxis] - self.yhat(theta)) ** 2
    return ((1/self.X.shape[1]) * sqrd_residuals.sum().values)[0]

  def del_cost_0(self, theta): 
    return np.divide(-2,self.N) * (self.y[:, np.newaxis] - self.yhat(theta)).sum()

  def del_cost_1(self, theta): 
    return np.divide(-2,self.N) * (self.X[self.feat][:, np.newaxis] * (self.y[:, np.newaxis] - self.yhat(theta))).sum()

# algorithm: repeat theta_k <- theta_k - alpha * del H / del theta_k `epochs` times, for k in range(len(theta))

  def update(self, theta): 
    
    return [theta[0] - self.gamma * self.del_cost_0(theta).values, theta[1] - self.gamma * self.del_cost_1(theta).values]

  def update_25x(self, the):
    return repeatedly(self.update, 25)(the)


test = GDOLSR(df_normalized, 'lweight', 'lpsa', 1000, gamma=0.0015)

test.cost(theta), test.cost(test.update_25x(test.theta)), test.cost(test.update_25x(test.update_25x(test.theta)))


theta_from_sklearn = [from_sklearn['intercept'], from_sklearn['coefficients'][0]]

test.cost(theta_from_sklearn), test.cost(test.update_25x(test.update_25x(test.theta)))

(62.42824056776314, 39.393366478746515)

In [0]:
# F : R -> R


In [0]:
def update(theta): 
  theta -= partial_cost_0,1
  return None

for k in range(epochs): 
  update(theta)
  

def update(theta): 
  return theta - partial_cost

In [238]:
(y[:, np.newaxis] - yhat(theta)).sum()

0    12.452598
dtype: float64

In [141]:
cost(theta)[0]

35.509789318731464

In [241]:
theta_from_sklearn = [from_sklearn['intercept'], from_sklearn['coefficients'][0]]

theta_ours = repeatedly(update, epochs//12)(theta)

#np.testing.assert_almost_equal(theta_ours, theta_from_sklearn)

list(map(lambda l: l[0], theta_ours)), theta_from_sklearn

cost(theta_ours), cost(theta_from_sklearn)

# in theory, sklearn's theta is already veyr good, so update won't do much at all
theta_from_sklearn, update(theta_from_sklearn) ## haha, there must be a mistake somewhere

([1.4138982670092684, 0.7678408275710042],
 [array([1.71155006]), array([0.45790337])])

In [253]:
dDD = {k: cost(F(theta)) for F in [repeatedly(update_25x, k) for k in range(1, 80)]}

## run once cuz it takes FOREVER. 
#alt.Chart(pd.DataFrame.from_dict(dDD.reset_index)).mark_line().encode(x='')

/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:32: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in subtract


In [252]:
cost(repeatedly(update_25x, 10)(theta))

/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:32: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in subtract


array([0.])

## Stretch Goals

If you happen upon the most useful resources for accomplishing this challenge first, I want you to spend time today studying other variations of Gradient Descent-Based Optimizers.

- Try and write a function that can perform gradient descent for arbitarily large (in dimensionality) multiple regression models. 
- Create a notebook for yourself exploring these topics
- How do they differ from the "vanilla" gradient descent we explored today
- How do these different gradient descent-based optimizers seek to overcome the challenge of finding the global minimum among various local minima?
- Write a blog post that reteaches what you have learned about these other gradient descent-based optimizers.

[Overview of GD-based optimizers](http://ruder.io/optimizing-gradient-descent/)

[Siraj Raval - Evolution of Gradient Descent-Based Optimizers](https://youtu.be/nhqo0u1a6fw)

# What you need to know about derivatives for this



In [261]:
class A: 
  def __init__(self, p, q, r=5): 
    self.p = p
    self.q = q
    self.r = r
    self.s = p ** q / r
  
  def getvals(self): 
    print(self.p, self.q, self.r, self.s)
    return (self.p,self.q,self.r,self.s)

aaaaaa = A(1,2)

aaaaaa.getvals()

1 2 5 0.2


(1, 2, 5, 0.2)